# Live Coding: Testing in MLOps con Prometheus e Grafana

**Obiettivo della lezione**: Imparare a monitorare un modello di Machine Learning in un ambiente di produzione simulato, utilizzando Prometheus per la raccolta di metriche e Grafana per la loro visualizzazione. Questo processo è una forma di **testing continuo** fondamentale nelle pratiche MLOps.

**Dataset**: Useremo il [Credit Card Fraud Detection Dataset](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud). Questo dataset è ideale perché è sbilanciato e ci permette di monitorare metriche interessanti come il numero di frodi rilevate.

**Prerequisiti software**:
1.  **Docker e Docker Compose**: Per eseguire Prometheus e Grafana in container isolati.
2.  **Python 3.8+**
3.  Le librerie Python che installeremo tra poco.
4.  Il file `creditcard.csv` scaricato e presente in questa cartella.

### Cella 1: Installazione delle librerie

Iniziamo installando le librerie necessarie per questo progetto.

In [ ]:
!pip install pandas scikit-learn==1.3.2 flask prometheus-client requests joblib

### Cella 2: Setup dell'Ambiente con Docker

Per eseguire Prometheus e Grafana, useremo Docker Compose. È il modo più semplice per gestire più container. 

**Passo 1: Creare i file di configurazione**

Nella stessa cartella di questo notebook, crea una sotto-cartella chiamata `prometheus`. All'interno di `prometheus`, crea un file chiamato `prometheus.yml` con il seguente contenuto:

```yaml
# prometheus/prometheus.yml
global:
  scrape_interval: 15s # Frequenza con cui raccogliere le metriche

scrape_configs:
  - job_name: 'ml-model-app'
    # L'host 'host.docker.internal' è una keyword speciale di Docker
    # che permette al container di raggiungere i servizi in esecuzione sulla macchina host (il tuo PC)
    static_configs:
      - targets: ['host.docker.internal:5000'] 
```

Successivamente, nella cartella principale (la stessa di questo notebook), crea un file chiamato `docker-compose.yml`:

```yaml
# docker-compose.yml
version: '3.7'

services:
  prometheus:
    image: prom/prometheus:latest
    container_name: prometheus
    ports:
      - "9090:9090"
    volumes:
      - ./prometheus:/etc/prometheus
    command:
      - '--config.file=/etc/prometheus/prometheus.yml'

  grafana:
    image: grafana/grafana:latest
    container_name: grafana
    ports:
      - "3000:3000"
    depends_on:
      - prometheus
```

**Passo 2: Avviare i container**

Apri un terminale nella cartella di questo notebook ed esegui:
```bash
docker-compose up -d
```

Ora hai Prometheus in esecuzione su `http://localhost:9090` e Grafana su `http://localhost:3000`.

### Cella 3: Addestramento del Modello di Machine Learning

Ora prepariamo il nostro modello. Useremo `scikit-learn` per addestrare un `LogisticRegression` sul dataset delle frodi. Il modello addestrato e lo `scaler` (usato per normalizzare i dati) verranno salvati su disco per essere usati dalla nostra API.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib

print("Caricamento del dataset... (potrebbe richiedere qualche istante)")
df = pd.read_csv('creditcard.csv')

print("Preparazione dei dati...")
X = df.drop('Class', axis=1)
y = df['Class']

# Normalizziamo i dati: è fondamentale per molti modelli, inclusa la Logistic Regression
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Divisione in training e test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

print("Addestramento del modello...")
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

print("Valutazione del modello sul test set:")
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

# Salvataggio del modello e dello scaler per l'uso nell'API
joblib.dump(model, 'model.joblib')
joblib.dump(scaler, 'scaler.joblib')

print("\nModello e scaler salvati come 'model.joblib' e 'scaler.joblib'")

### Cella 4: Creare un'API per il Modello con Flask

Ora che abbiamo un modello, dobbiamo "servirlo" tramite un'API. Useremo **Flask**. La parte cruciale è l'**instrumentazione**: aggiungeremo codice per esporre le metriche che Prometheus potrà leggere.

**Metriche che tracceremo**:
1.  `predictions_total`: Contatore del numero totale di predizioni (con etichetta `fraud`/`non_fraud`).
2.  `prediction_latency_seconds`: Istogramma per misurare il tempo di risposta.
3.  `model_accuracy`: Un *gauge* (misuratore) che potremmo aggiornare per monitorare l'accuratezza.

**Azione richiesta**: Salva il codice seguente in un file separato chiamato `app.py` nella stessa cartella di questo notebook.

```python
# app.py
import joblib
import pandas as pd
import time
from flask import Flask, request, jsonify
from prometheus_client import Counter, Histogram, make_wsgi_app

# 1. Inizializza l'applicazione Flask
app = Flask(__name__)

# 2. Carica il modello e lo scaler addestrati
model = joblib.load('model.joblib')
scaler = joblib.load('scaler.joblib')

# 3. Definisci le metriche di Prometheus
PREDICTIONS_TOTAL = Counter(
    'predictions_total', # Nome della metrica
    'Total number of predictions made',
    ['class_name'] # Etichetta per distinguere le classi
)
PREDICTION_LATENCY = Histogram(
    'prediction_latency_seconds',
    'Time taken to process a prediction request'
)

# 4. Crea l'endpoint di predizione `/predict`
@app.route('/predict', methods=['POST'])
def predict():
    start_time = time.time()
    
    # Prendi i dati JSON dalla richiesta
    data = request.get_json()
    df = pd.DataFrame(data, index=)
    
    # Applica lo scaler (DEVE essere lo stesso usato in addestramento)
    data_scaled = scaler.transform(df)
    
    # Esegui la predizione
    prediction = model.predict(data_scaled)
    class_name = "fraud" if prediction == 1 else "non_fraud"
    
    # Aggiorna il contatore delle predizioni
    PREDICTIONS_TOTAL.labels(class_name=class_name).inc()
        
    # Calcola e registra la latenza
    latency = time.time() - start_time
    PREDICTION_LATENCY.observe(latency)
    
    return jsonify({'prediction': int(prediction), 'class_name': class_name})

# 5. Esponi l'endpoint `/metrics` per Prometheus
# La libreria prometheus_client si occupa di creare questo endpoint per noi
from werkzeug.middleware.dispatcher import DispatcherMiddleware
app.wsgi_app = DispatcherMiddleware(app.wsgi_app, {
    '/metrics': make_wsgi_app()
})

# 6. Avvia l'applicazione
if __name__ == '__main__':
    # Esegui su porta 5000, accessibile da tutte le interfacce di rete ('0.0.0.0')
    app.run(host='0.0.0.0', port=5000)
```

**Passo successivo**: Apri un **nuovo terminale**, vai nella cartella del progetto ed esegui:
```bash
python app.py
```
Se tutto è corretto, il server Flask è in esecuzione. Puoi verificare che le metriche siano esposte visitando `http://localhost:5000/metrics` nel tuo browser (al momento saranno a zero).

### Cella 5: Visualizzazione con Grafana

È il momento di visualizzare! Andremo sull'interfaccia di Grafana e costruiremo una dashboard.

**Passo 1: Accedi a Grafana**
- Apri `http://localhost:3000`.
- **Username**: admin, **Password**: admin (ti chiederà di cambiarla).

**Passo 2: Aggiungi Prometheus come Data Source**
1.  Menu a sinistra (icona ingranaggio) -> **Data Sources** -> **Add data source**.
2.  Seleziona **Prometheus**.
3.  URL: `http://prometheus:9090`. (Usiamo `prometheus` come hostname perché i container sono nella stessa rete Docker).
4.  Clicca **Save & Test**.

**Passo 3: Crea una Dashboard**
1.  Menu a sinistra (icona 4 quadrati) -> **Dashboards** -> **New Dashboard** -> **Add visualization**.

**Pannello 1: Latenza delle Predizioni (95° percentile)**
- **Query PromQL**: `histogram_quantile(0.95, sum(rate(prediction_latency_seconds_bucket[5m])) by (le))`
- **Titolo**: `Latenza Predizioni (P95)`
- **Opzioni standard -> Unit**: `seconds (s)`

**Pannello 2: Richieste al secondo (Throughput)**
- **Query PromQL**: `sum(rate(predictions_total[5m]))`
- **Titolo**: `Throughput (req/sec)`

**Pannello 3: Ripartizione Predizioni (Frode vs Non Frode)**
- **Query PromQL**: `sum(rate(predictions_total[5m])) by (class_name)`
- **Tipo di visualizzazione**: `Pie Chart` (Grafico a torta)
- **Titolo**: `Ripartizione Frodi`

**Salva la dashboard!** Clicca sull'icona del dischetto in alto a destra.

### Cella 6: Simulare Traffico e Osservare la Dashboard

La dashboard è pronta ma vuota. Creiamo uno script Python per simulare degli utenti che inviano dati alla nostra API. Eseguiremo questa cella direttamente dal notebook per generare traffico.

Mentre la cella seguente è in esecuzione, **aggiorna la tua dashboard di Grafana**: vedrai i grafici popolarsi in tempo reale!

In [ ]:
import requests
import pandas as pd
import time
import random

# Carichiamo i dati che useremo per la simulazione (escludendo la colonna target 'Class')
try:
    simulation_data = pd.read_csv('creditcard.csv').drop('Class', axis=1)
except FileNotFoundError:
    print("ERRORE: File 'creditcard.csv' non trovato. Scaricalo da Kaggle e mettilo nella stessa cartella.")
    simulation_data = None

if simulation_data is not None:
    print("Avvio della simulazione... Invierò una richiesta ogni 0.5-2 secondi.")
    print("Premi 'Interrupt' (il quadrato nero) nel menù di Jupyter per fermare.")

    api_url = 'http://localhost:5000/predict'
    headers = {'Content-Type': 'application/json'}

    try:
        while True:
            # Scegli una transazione a caso dal dataset
            sample = simulation_data.sample(1)
            
            # Convertila in formato JSON (come richiesto dall'API)
            payload = sample.to_dict(orient='records')[0]

            try:
                response = requests.post(api_url, json=payload, headers=headers)
                if response.status_code == 200:
                    result = response.json()
                    print(f"Richiesta inviata. Risposta: {result['class_name']}")
                else:
                    print(f"Errore dall'API: {response.status_code}")
            except requests.exceptions.ConnectionError:
                print("Errore di connessione. L'app Flask è in esecuzione? Interrompo la simulazione.")
                break

            # Aspetta un tempo casuale prima della prossima richiesta
            time.sleep(random.uniform(0.5, 2.0))
            
    except KeyboardInterrupt:
        print("\nSimulazione interrotta dall'utente.")

### Cella 7: Discussione e Conclusioni

Abbiamo costruito una pipeline di monitoraggio di base. Riassumiamo cosa abbiamo fatto e perché è importante per il "Testing in MLOps".

1.  **Abbiamo addestrato e servito un modello**: Questo è il primo passo di MLOps: rendere il modello utilizzabile.
2.  **Abbiamo instrumentato l'API**: Abbiamo aggiunto metriche per avere visibilità sul suo comportamento.
3.  **Abbiamo visualizzato le metriche**: Abbiamo dato un senso ai dati grezzi con una dashboard.

**Come si collega al Testing?**

Questo è **testing continuo in produzione**. Le metriche ci aiutano a rispondere a domande critiche:

- **Il servizio è funzionante? (Health Check)**
  - La metrica `up` in Prometheus (query: `up{job="ml-model-app"}`) ci dice se l'app risponde. Il nostro grafico di *Throughput* è un altro ottimo indicatore di salute.

- **Il servizio è performante? (Performance Testing)**
  - La *Latenza* è la metrica chiave. Se sale, l'esperienza utente peggiora. Possiamo impostare un alert se supera una certa soglia (es. 500ms).

- **Il modello si comporta in modo anomalo? (Model Behavior Testing)**
  - La *Ripartizione delle Predizioni* è un potente indicatore di **data drift**. Se in addestramento le frodi erano lo 0.17% e in produzione iniziamo a prevederne il 10%, i dati in input sono probabilmente cambiati e il modello potrebbe non essere più valido. Questo è un segnale per investigare e, se necessario, ri-addestrare.

**Passi Successivi in un Sistema Reale**

1.  **Alerting**: Configurare alert automatici in Grafana o Alertmanager (es. "Se la latenza P95 > 0.5s per 5 minuti, invia una notifica su Slack").
2.  **Monitoraggio della Qualità Reale (Accuracy, Precision, etc.)**: Richiede un meccanismo di feedback per ottenere il "ground truth". Questo è un task più complesso che implica salvare le predizioni e confrontarle con i dati reali quando diventano disponibili.
3.  **A/B Testing**: Si possono deployare due versioni del modello e aggiungere un'etichetta `version` alle metriche per confrontare le loro performance in tempo reale sulla stessa dashboard.